In [1]:
# Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
sys.path.insert(1, r'C:\Users\soube\OneDrive\Desktop\Hammudi\Bachelorarbeit\Repository\AP-rent-determination\tree_models\tree_model_functions.py')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Selecting the DataSource
dataSource = r"C:\Users\soube\OneDrive\Desktop\Hammudi\Bachelorarbeit\Repository\AP-rent-determination\immonet_data\cleaned_data_without_NaN.csv"

# Selecting columns to drop out of featureList and creating LabelList
featureDropList = ["_id", "observationDate", "state", "city", "AP_community", "base_rent"]
LabelList = ["base_rent"]

# Create DataFrame from DataSource
dataframe = pd.read_csv(dataSource)

# Create list of unique states
states = dataframe["state"].unique()

# Create list for model scores
state_prediction_score = []

# Create list for most important features
feature_importances = []

In [4]:
# Loop throug all states to train them seperately
for state in states:
    df = dataframe
    df = df[df["state"]  == state]

    # Create feature and label lists
    y = df[LabelList]
    X = df.drop(featureDropList, axis = 1)
    feature_list = list(X.columns)

    y = np.array(y)
    X = np.array(X)

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

    # Instantiate model
    hgbr = HistGradientBoostingRegressor(random_state=0)

    # Train the model on training data
    hgbr.fit(X_train, y_train)

    # Use the Regressors's predict method on the test data
    predictions = hgbr.predict(X_test)

    # Calculate the absolute errors
    errors = abs(predictions - y_test)

    # Print out the mean absolute error (mae)
    #print('Average model error:', round(np.mean(errors), 2), 'euros.')
    
    # Calculate relative prediction errors
    errors = [100 * (abs(predictions[i] - y_test[i])/ y_test[i]) for i in range(min(len(predictions), len(y_test)))]

    # Count of predictions that are at least 10% accurate
    count_good_predictions = sum(1 for i in errors if i <= 10)

    # Proportion of good predictions for the Testset
    good_predictions = round(np.mean(100 * (count_good_predictions / len(errors))), 2)
    #print('Percentage of predictions with less than 10 % deviation: ', good_predictions, '%.')

    # Hier muss Code für die importances stehen
    ##########
    
    state_prediction_score.append([state, good_predictions])

c:\Users\soube\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\soube\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\soube\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\soube\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConve

In [5]:
print(state_prediction_score)

[['Schleswig-Holstein', 77.85], ['Bremen', 58.43], ['Hessen', 68.86], ['Hamburg', 76.68], ['Nordrhein-Westfalen', 62.49], ['Baden-Württemberg', 58.95], ['Bayern', 65.01], ['Thüringen', 66.61], ['Mecklenburg-Vorpommern', 70.5], ['Sachsen', 70.46], ['Brandenburg', 66.78], ['Berlin', 72.93], ['Saarland', 60.58], ['Sachsen-Anhalt', 68.94], ['Niedersachsen', 69.89], ['Rheinland-Pfalz', 59.51]]


In [6]:
# Convert state_prediction_score list into DataFrame
state_prediction_df = pd.DataFrame(state_prediction_score, columns = ["state", "score"])
# Calculate the average score of the model
prediction_score = state_prediction_df["score"].mean()
print("Model score :", prediction_score, "%")

Model score : 67.154375 %


### Get rid of unnnecessary Variables ###